# POD and fft from pointclouds 

We will now proceed to explain how to perform POD from point clouds. In this instance, we test only for POD in serial, as to perform in parallel, a parallel reader/writer is needed.

If you have saved information in hdf5 and have habilitated mpi4py compilation of it, then you could use this code in parallel.

#### Import general modules

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np
import h5py
import sys

# Get mpi info
comm = MPI.COMM_WORLD

## Set up the input parameters

In [2]:
file_sequence = [f"interpolated_fields{str(1+i).zfill(5)}.hdf5" for i in range(0, 48)]
#file_sequence = [f"interpolated_fields{str(1+i).zfill(5)}.hdf5" for i in range(0, 5)]
#file_sequence = [f"interpolated_fields{str(1+i).zfill(5)}.hdf5" for i in range(0, 1)]
pod_fields = ["u", "v", "w"]
mesh_fname = "points.hdf5"
mass_matrix_fname = "points.hdf5"
mass_matrix_key = "mass"
k = len(file_sequence)
p = len(file_sequence)
fft_axis = 1 # 0 for x, 1 for y, 2 for z (Depends on how the mesh was created)

# Calculated parameters
number_of_pod_fields = len(pod_fields)

## Call the pynektools routines

In [3]:
from pynektools.rom.pod import POD
from pynektools.rom.io_help import IoHelp
# ================
# Helper functions
# ================

def get_wavenumber_slice(kappa, fft_axis):
    if fft_axis == 0:
        return (kappa, slice(None), slice(None))
    elif fft_axis == 1:
        return (slice(None), kappa, slice(None))
    elif fft_axis == 2:
        return (slice(None), slice(None), kappa)

def get_mass_slice(fft_axis):
    # Have a slice of the axis to perform the fft
    if fft_axis == 0:
        mass_slice = (0, slice(None), slice(None))
    elif fft_axis == 1:
        mass_slice = (slice(None), 0, slice(None))
    elif fft_axis == 2:
        mass_slice = (slice(None), slice(None), 0)
    return mass_slice

def get_2d_slice_shape(fft_axis, field_shape):
    if fft_axis == 0:
        return (field_shape[1], field_shape[2])
    elif fft_axis == 1:
        return (field_shape[0], field_shape[2])
    elif fft_axis == 2:
        return (field_shape[0], field_shape[1])

def fourier_normalization(N_samples):
    return np.sqrt(N_samples)

def degenerate_scaling(kappa):
    if kappa == 0:
        scaling = 1
    else:
        scaling = 2
    return np.sqrt(scaling)

def physical_space(pod: dict[int, POD], ioh: dict[int, IoHelp], wavenumbers: list[int], modes: list[int], field_shape: tuple, fft_axis: int, field_names: list[str], N_samples: int, snapshots: list[int] = None):
    """
    Function to transform modes or snapshots from the POD objects into the physical space

    This will either produce a set of specified modes for specified wavenumbers in physical space.

    Or it will use the specified modes in the specified wavenumbers to reconstruct the specified snapshots in physical space.

    Parameters
    ----------
    pod : dict[int, POD]
        Dictionary of POD object with the modes to transform to physical space
        the int key is the wavenumber
    ioh : dict[int, IoHelp]
        Dictionary of IoHelp object, which has some functionalities to split fields
        the int key is the wavenumber    
    wavenumbers : list[int]
        List of wavenumbers to use in the operations
    modes : int
        list of the modes to use in the operations.
        if snapshot is not given, the modes will be transformed to physical space and returned.
        if snapshot is given, the modes will be used to reconstruct the snapshots and returned.
    field_shape : tuple
        Shape of the field in physical space
    fft_axis : int
        Axis where the fft was performed
    field_names : list[str]
        List of field names to put in the output dictionary
    N_samples : int
        Number of samples in the fft
    snapshots : list[int], optional
        List of snapshots to transform to physical space, by default None
        If this option is given, then the return will be a list of snapshots in physical space
        using the snapshot indices for the reconstruction.
        Be mindfull that the snapshot indices should be in the range of the snapshots used to create the POD objects.

    Returns
    -------
    """

    # To reconstruct snapshots
    if isinstance(snapshots, list):

        physical_fields = {}

        # Reconstruct the fourier coefficients per wavenumber with the given snapshots and modes
        fourier_reconstruction = {}
        for kappa in wavenumbers:
            fourier_reconstruction[kappa] = pod[kappa].u_1t[:,modes].reshape(-1, len(modes)) @ np.diag(pod[kappa].d_1t[modes]) @ pod[kappa].vt_1t[np.ix_(modes,snapshots)].reshape(len(modes), len(snapshots))
    
        # Go thorugh the wavenumbers in the list and put the modes in the physical space
        for snapshot in snapshots:

            physical_fields[snapshot] = {}

            # Create a buffer to zero out all the other wavenumber contributions
            fourier_field_3d = [np.zeros(field_shape, dtype=pod[0].u_1t.dtype) for i in range(0, number_of_pod_fields)]

            # Fill the fourier fields with the contributions of the wavenumbers
            for kappa in wavenumbers:

                ## Split the 1d snapshot into a list with the fields you want
                field_list1d = ioh[kappa].split_narray_to_1dfields(fourier_reconstruction[kappa][:,snapshot])
                ## Reshape the obtained 1d fields to be 2d
                _2d_field_shape = get_2d_slice_shape(fft_axis, field_shape)
                field_list_2d = [field.reshape(_2d_field_shape) for field in field_list1d]
            
                # Get the proper data slice for positive and negative wavenumber
                positive_wavenumber_slice = get_wavenumber_slice(kappa, fft_axis)
                negative_wavenumber_slice = get_wavenumber_slice(-kappa, fft_axis)
                
                for i, field_name in enumerate(field_names):
                 
                    # Fill the buffer with the proper wavenumber contribution
                    fourier_field_3d[i][positive_wavenumber_slice] = field_list_2d[i]
                    if kappa != 0:
                        fourier_field_3d[i][negative_wavenumber_slice] = np.conj(field_list_2d[i])

            for i, field_name in enumerate(field_names):

                # Perform the inverse fft
                physical_field_3d = np.fft.ifft(fourier_field_3d[i] * fourier_normalization(N_samples), axis = fft_axis) # Rescale the coefficients
                
                # Save the field in the dictionary (only the real part)
                physical_fields[snapshot][field_name] = np.copy(physical_field_3d.real)

    # To obtain only the modes
    else:

        # Go thorugh the wavenumbers in the list and put the modes in the physical space
        physical_fields = {}
        for kappa in wavenumbers:

            # Create the physical space dictionary
            physical_fields[kappa] = {}

            for mode in modes:

                # Add the mode to the dictionary
                physical_fields[kappa][mode] = {}

                ## Split the 1d snapshot into a list with the fields you want
                field_list1d = ioh[kappa].split_narray_to_1dfields(pod[kappa].u_1t[:,mode])
                ## Reshape the obtained 1d fields to be 2d
                _2d_field_shape = get_2d_slice_shape(fft_axis, field_shape)
                field_list_2d = [field.reshape(_2d_field_shape) for field in field_list1d]
            
                # Get the proper data slice for positive and negative wavenumber
                positive_wavenumber_slice = get_wavenumber_slice(kappa, fft_axis)
                negative_wavenumber_slice = get_wavenumber_slice(-kappa, fft_axis)
                
                for i, field_name in enumerate(field_names):
                 
                    # Create a buffer to zero out all the other wavenumber contributions
                    fourier_field_3d = np.zeros(field_shape, dtype=pod[kappa].u_1t.dtype)
                
                    # Fill the buffer with the proper wavenumber contribution
                    fourier_field_3d[positive_wavenumber_slice] = field_list_2d[i]
                    if kappa != 0:
                        fourier_field_3d[negative_wavenumber_slice] = np.conj(field_list_2d[i])

                    # Perform the inverse fft
                    physical_field_3d = np.fft.ifft(fourier_field_3d * fourier_normalization(N_samples), axis = fft_axis) # Rescale the coefficients
                    
                    # Save the field in the dictionary (only the real part)
                    physical_fields[kappa][mode][field_name] = np.copy(physical_field_3d.real)
       
    return physical_fields

# ============
# Main program
# ============
# Initialize
# ============

# Import IO helper functions
from pynektools.io.utils import get_fld_from_ndarray

# Import types asociated with POD
from pynektools.rom.pod import POD
from pynektools.rom.io_help import IoHelp

# Output
from pyevtk.hl import gridToVTK

# Load the mesh
with h5py.File(mesh_fname, 'r') as f:
    x = f["x"][:]
    y = f["y"][:]
    z = f["z"][:]

# Load the mass matrix
with h5py.File(mass_matrix_fname, 'r') as f:
    bm = f[mass_matrix_key][:]
bm[np.where(bm == 0)] = 1e-14
_3d_bm_shape = bm.shape

# Obtain the number of frequencies you will obtain
N_samples = bm.shape[fft_axis]
number_of_frequencies = N_samples // 2 + 1
# Choose the proper mass matrix slice
bm = bm[get_mass_slice(fft_axis)]

ioh = {"wavenumber" : "buffers"}
pod = {"wavenumber" : "POD object"}

# Initialize the buffers and objects for each wavenumber
for kappa in range(0, number_of_frequencies):

    # Instance io helper that will serve as buffer for the snapshots
    ioh[kappa] = IoHelp(comm, number_of_fields = number_of_pod_fields, batch_size = p, field_size = bm.size, mass_matrix_data_type=bm.dtype, field_data_type=np.complex128,  module_name="buffer_kappa"+str(kappa))

    # Put the mass matrix in the appropiate format (long 1d array)
    mass_list = []
    for i in range(0, number_of_pod_fields):
        mass_list.append(np.copy(np.sqrt(bm)))
    ioh[kappa].copy_fieldlist_to_xi(mass_list)
    ioh[kappa].bm1sqrt[:,:] = np.copy(ioh[kappa].xi[:,:])

    # Instance the POD object
    pod[kappa] = POD(comm, number_of_modes_to_update = k, global_updates = True, auto_expand = False)

# ============
# Main program
# ============
# Update modes
# ============

j = 0
while j < len(file_sequence):

    # Load the snapshot data
    fname = file_sequence[j]
    with h5py.File(fname, 'r') as f:
        fld_data = []
        for field in pod_fields:
            fld_data.append(f[field][:])

    # Perform the fft
    for i in range(0, number_of_pod_fields):
        fld_data[i] = np.fft.fft(fld_data[i], axis = fft_axis)/fourier_normalization(N_samples)

    # For each wavenumber, load buffers and update if needed   
    for kappa in range(0, number_of_frequencies):

        # Get the proper slice for the wavenumber
        positive_wavenumber_slice = get_wavenumber_slice(kappa, fft_axis)

        # Get the wavenumber data
        wavenumber_data = []
        for i in range(0, number_of_pod_fields):
            wavenumber_data.append(fld_data[i][positive_wavenumber_slice]*degenerate_scaling(kappa)) # Here add contributions from negative wavenumbers

        # Put the fourier snapshot data into a column array
        ioh[kappa].copy_fieldlist_to_xi(wavenumber_data)

        # Load the column array into the buffer
        ioh[kappa].load_buffer(scale_snapshot = True)
        
        # Update POD modes
        if ioh[kappa].update_from_buffer:
            pod[kappa].update(comm, buff = ioh[kappa].buff[:,:(ioh[kappa].buffer_index)])

    j += 1

# ============
# Main program
# ============
# rscale modes
# ============

for kappa in range(0, number_of_frequencies):
    # Check if there is information in the buffer that should be taken in case the loop exit without flushing
    if ioh[kappa].buffer_index > ioh[kappa].buffer_max_index:
        ioh[kappa].log.write("info","All snapshots where properly included in the updates")
    else: 
        ioh[kappa].log.write("warning","Last loaded snapshot to buffer was: "+repr(ioh[kappa].buffer_index-1))
        ioh[kappa].log.write("warning","The buffer updates when it is full to position: "+repr(ioh[kappa].buffer_max_index))
        ioh[kappa].log.write("warning","Data must be updated now to not lose anything,  Performing an update with data in buffer ")
        pod[kappa].update(comm, buff = ioh[kappa].buff[:,:(ioh[kappa].buffer_index)])

    # Scale back the modes (with the mass matrix)
    pod[kappa].scale_modes(comm, bm1sqrt = ioh[kappa].bm1sqrt, op = "div")

    # Scale back the modes (with wavenumbers and degeneracy)
    pod[kappa].u_1t = pod[kappa].u_1t/degenerate_scaling(kappa)

    # Rotate local modes back to global, This only enters in effect if global_update = false
    pod[kappa].rotate_local_modes_to_global(comm)

# ============
# Main program
# ============
# Writing data
# ============

if 1==0:

    for kappa in range(0, number_of_frequencies):

        for j in range(0, k):
            
            ## Split the 1d snapshot into a list with the fields you want
            field_list1d = ioh[kappa].split_narray_to_1dfields(pod[kappa].u_1t[:,j])
            ## Reshape the obtained 1d fields to be 2d
            field_list_2d = [field.reshape(bm.shape) for field in field_list1d]
            
            field_dict = {}
            positive_wavenumber_slice = get_wavenumber_slice(kappa, fft_axis)
            negative_wavenumber_slice = get_wavenumber_slice(-kappa, fft_axis)
            for i in range(0, len(pod_fields)):
            
                # Create a buffer to zero out all the other wavenumber contributions
                fourier_field_3d = np.zeros(_3d_bm_shape, dtype=pod[kappa].u_1t.dtype)
            
                # Fill the buffer with the proper wavenumber contribution
                fourier_field_3d[positive_wavenumber_slice] = field_list_2d[i]
                if kappa != 0:
                    fourier_field_3d[negative_wavenumber_slice] = np.conj(field_list_2d[i])

                # Perform the inverse fft
                #physical_field_3d = np.fft.ifft(fourier_field_3d*N_samples/multiplier, axis = fft_axis) # Rescale the coefficients
                physical_field_3d = np.fft.ifft(fourier_field_3d*fourier_normalization(N_samples), axis = fft_axis) # The values have previously been rescaled so no need to do it again
                
                # Save the field in the dictionary later used to write the vtk
                field_dict[f"{pod_fields[i]}_mode"] = np.copy(physical_field_3d.real)

            # write to vtk
            gridToVTK( f"kappa{kappa}_pod_mode"+str(j).zfill(5),  x, y, z, pointData=field_dict)

2024-09-25 13:17:27,693 - buffer_kappa0 - INFO - io_helper object initialized
2024-09-25 13:17:27,695 - pod - WARNING - ifget_all_modes is hard coded to False. This parameter applies to lcl updates. It controls if one gets all modes in the global rotation, despite keeping less modes locally. I do not see a use for this in production runs. Thus it is set to false. If needed, activate in mpi_spSVD.py module
2024-09-25 13:17:27,696 - pod - INFO - POD Object initialized
2024-09-25 13:17:27,697 - buffer_kappa1 - INFO - io_helper object initialized
2024-09-25 13:17:27,698 - pod - WARNING - ifget_all_modes is hard coded to False. This parameter applies to lcl updates. It controls if one gets all modes in the global rotation, despite keeping less modes locally. I do not see a use for this in production runs. Thus it is set to false. If needed, activate in mpi_spSVD.py module
2024-09-25 13:17:27,698 - pod - INFO - POD Object initialized
2024-09-25 13:17:27,699 - buffer_kappa2 - INFO - io_helper

/tmp/ipykernel_11162/3377758638.py:220: ComplexWarning: Casting complex values to real discards the imaginary part
  ioh[kappa].bm1sqrt[:,:] = np.copy(ioh[kappa].xi[:,:])


2024-09-25 13:17:27,897 - buffer_kappa0 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,899 - buffer_kappa1 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,900 - buffer_kappa2 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,900 - buffer_kappa3 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,901 - buffer_kappa4 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,902 - buffer_kappa5 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,902 - buffer_kappa6 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,903 - buffer_kappa7 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,904 - buffer_kappa8 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,905 - buffer_kappa9 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,907 - buffer_kappa10 - INFO - Loaded snapshot in buffer in pos: 4
2024-09-25 13:17:27,907 - buffer_kappa11 - INFO - Loaded snapsho

In [4]:
phys = physical_space(pod, ioh, wavenumbers=[k for k in range(0, number_of_frequencies)], modes=[i for i in range(0, k)], field_shape=_3d_bm_shape, fft_axis=fft_axis, field_names=pod_fields, N_samples=N_samples, snapshots=[i for i in range(0, len(file_sequence))])

## Perform tests

First check that the reconstruction of every snapshot matches and that the energy is kept in the time coefficients, not in the modes.

In [5]:
# Load the mass matrix
with h5py.File(mass_matrix_fname, 'r') as f:
    bm_v = f["mass"][:]

# Go through the snapshots in the file sequence
total_snapshot_energy = 0
for j, fname in enumerate(file_sequence):
    print(f"Testing snapshot {j}: {fname}")
    
    with h5py.File(fname, 'r') as f:
        
        # Check if the reconstruction is accurate
        for field in pod_fields:
            passed = np.allclose(phys[j][field], f[field][:])
            print(f"Field {field} passed: {passed}")

        # Check if the energy was accurately captured
        snapshot_energy = 0
        for field in pod_fields:
            snapshot_energy += np.sum(f[field][:]**2*bm_v)
        total_snapshot_energy += snapshot_energy

        reconstruction_energy = 0
        for kappa in range(0, number_of_frequencies):
            a_i = np.diag(pod[kappa].d_1t)@pod[kappa].vt_1t[:,j]
            reconstruction_energy += np.sum(np.abs(a_i)**2)

        print(f"Snapshot energy: {snapshot_energy}")
        print(f"Reconstruction energy: {reconstruction_energy}")
    
    print("=======================================")

# Check if the total energy is kept in the singular values
print(f"total snapshot energy: {total_snapshot_energy}")

total_pod_energy = 0
for kappa in range(0, number_of_frequencies):
    total_pod_energy += np.sum(pod[kappa].d_1t**2)
print(f"Total POD energy: {total_pod_energy}")
print("=======================================")   

Testing snapshot 0: interpolated_fields00001.hdf5
Field u passed: True
Field v passed: True
Field w passed: True
Snapshot energy: 3.698958237844183e-05
Reconstruction energy: 3.6993163022397004e-05
Testing snapshot 1: interpolated_fields00002.hdf5
Field u passed: True
Field v passed: True
Field w passed: True
Snapshot energy: 3.8898814924962444e-05
Reconstruction energy: 3.8902239959461035e-05
Testing snapshot 2: interpolated_fields00003.hdf5
Field u passed: True
Field v passed: True
Field w passed: True
Snapshot energy: 4.153454569587408e-05
Reconstruction energy: 4.153817337208489e-05
Testing snapshot 3: interpolated_fields00004.hdf5
Field u passed: True
Field v passed: True
Field w passed: True
Snapshot energy: 4.4425076415412405e-05
Reconstruction energy: 4.442864337674737e-05
Testing snapshot 4: interpolated_fields00005.hdf5
Field u passed: True
Field v passed: True
Field w passed: True
Snapshot energy: 4.688317219673442e-05
Reconstruction energy: 4.688600091533262e-05
Testing sna

Then check if the left and right singular vectors are indeed orthogonal with respect to the others along the wavenumbers

In [8]:
# Check the left singular vectors
all_passed = True
for kappa in range(0, number_of_frequencies):
    
    kappa_i = 0
    kappa_j = kappa_i # These are the modes for the fourier coefficients. Thus, they orthogonals in a wave number. The fourier modes are orthogonal among fourier modes

    for i in range(0, k):
        for j in range(0, k):

            passed = True

            scaled_mode_i = pod[kappa_i].u_1t[:,i]*ioh[kappa_i].bm1sqrt[:,0]*degenerate_scaling(kappa_i)
            scaled_mode_j = pod[kappa_j].u_1t[:,j]*ioh[kappa_j].bm1sqrt[:,0]*degenerate_scaling(kappa_j)

            norm = np.abs(scaled_mode_i.T@scaled_mode_j)

            if (j==i) and (np.allclose(norm, 1)):
                passed = True
            elif (j!=i) and (np.allclose(norm, 0)):
                passed = True
            else:
                passed = False

            if not passed:              
                print(f"Mode {i} and mode {j} in wavenumber {kappa} are not orthogonal")
                print(f"Norm: {norm}")
                all_passed = False
                break

print(f"The modes are orthogonal: {all_passed}")

# Check the right singular vectors
all_passed = True
for kappa in range(0, number_of_frequencies):
    
    kappa_i = 0
    kappa_j = kappa_i # These are the modes for the fourier coefficients. Thus, they orthogonals in a wave number. The fourier modes are orthogonal among fourier modes

    for i in range(0, k):
        for j in range(0, k):

            passed = True

            scaled_mode_i = pod[kappa_i].vt_1t[i,:]
            scaled_mode_j = pod[kappa_j].vt_1t[j,:]

            norm = np.abs(scaled_mode_i.T@scaled_mode_j)

            if (j==i) and (np.allclose(norm, 1)):
                passed = True
            elif (j!=i) and (np.allclose(norm, 0)):
                passed = True
            else:
                passed = False

            if not passed:              
                print(f"Right singular vector {i} and right singular vector {j} in wavenumber {kappa} are not orthogonal")
                print(f"Norm: {norm}")
                all_passed = False
                break

print(f"The right singular vectors are orthogonal: {all_passed}")

The modes are orthogonal: True
The right singular vectors are orthogonal: True
